In [1]:
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
documents = dataset.data
len(documents)

11314

In [2]:
print(dataset.target_names)

['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


In [3]:
news_df = pd.DataFrame({'document':documents})
# 특수 문자 제거
news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z]", " ")
# 길이가 3이하인 단어는 제거 (길이가 짧은 단어 제거)
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
# 전체 단어에 대한 소문자 변환
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: x.lower())

In [9]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english') # NLTK로부터 불용어를 받아옵니다.
tokenized_doc = news_df['clean_doc'].apply(lambda x: x.split()) # 토큰화
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])
# 불용어를 제거합니다.

In [10]:
print(tokenized_doc[1])

['yeah', 'expect', 'people', 'read', 'actually', 'accept', 'hard', 'atheism', 'need', 'little', 'leap', 'faith', 'jimmy', 'logic', 'runs', 'steam', 'sorry', 'pity', 'sorry', 'feelings', 'denial', 'faith', 'need', 'well', 'pretend', 'happily', 'ever', 'anyway', 'maybe', 'start', 'newsgroup', 'atheist', 'hard', 'bummin', 'much', 'forget', 'flintstone', 'chewables', 'bake', 'timmons']


In [13]:
from gensim import corpora
dictionary = corpora.Dictionary(tokenized_doc)
corpus = [dictionary.doc2bow(text) for text in tokenized_doc]
print(corpus[1]) # 수행된 결과에서 두번째 뉴스 출력. 첫번째 문서의 인덱스는 0

[(52, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 2), (67, 1), (68, 1), (69, 1), (70, 1), (71, 2), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 2), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 2), (86, 1), (87, 1), (88, 1), (89, 1)]


In [22]:
print(dictionary[2])

austria


In [15]:
len(dictionary)

64281

In [16]:
import gensim
NUM_TOPICS = 20 #20개의 토픽, k=20
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.025*"game" + 0.022*"team" + 0.018*"year" + 0.017*"games"')
(1, '0.029*"drive" + 0.023*"disk" + 0.022*"windows" + 0.021*"card"')
(2, '0.015*"would" + 0.014*"people" + 0.008*"think" + 0.007*"like"')
(3, '0.014*"president" + 0.007*"ground" + 0.007*"printf" + 0.007*"char"')
(4, '0.031*"window" + 0.015*"widget" + 0.012*"display" + 0.011*"application"')
(5, '0.048*"file" + 0.036*"output" + 0.035*"entry" + 0.017*"section"')
(6, '0.029*"israel" + 0.025*"jews" + 0.019*"turkish" + 0.018*"israeli"')
(7, '0.014*"georgia" + 0.013*"rockefeller" + 0.012*"island" + 0.012*"homosexual"')
(8, '0.011*"available" + 0.009*"file" + 0.009*"information" + 0.008*"program"')
(9, '0.015*"said" + 0.014*"went" + 0.011*"period" + 0.008*"came"')
(10, '0.020*"armenian" + 0.011*"russian" + 0.010*"armenians" + 0.009*"university"')
(11, '0.035*"space" + 0.013*"nasa" + 0.012*"science" + 0.008*"earth"')
(12, '0.014*"encryption" + 0.013*"public" + 0.012*"security" + 0.011*"government"')
(13, '0.017*"health" + 0.013*"

In [18]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)
pyLDAvis.display(vis)

In [19]:
for i, topic_list in enumerate(ldamodel[corpus]):
    if i==5:
        break
    print(i,'번째 문서의 topic 비율은',topic_list)

0 번째 문서의 topic 비율은 [(2, 0.5490493), (6, 0.10314918), (9, 0.021023352), (10, 0.12575331), (13, 0.044226), (16, 0.14529912)]
1 번째 문서의 topic 비율은 [(2, 0.40071866), (3, 0.028745145), (4, 0.099006236), (11, 0.089249924), (19, 0.36251798)]
2 번째 문서의 topic 비율은 [(2, 0.77140343), (6, 0.17291965), (13, 0.04193159)]
3 번째 문서의 topic 비율은 [(2, 0.34759477), (3, 0.058244918), (12, 0.29636315), (13, 0.018582854), (14, 0.26742998)]
4 번째 문서의 topic 비율은 [(0, 0.37567255), (2, 0.2707884), (5, 0.076085895), (13, 0.2478137)]


In [20]:
def make_topictable_per_doc(ldamodel, corpus):
    topic_table = pd.DataFrame()

    # 몇 번째 문서인지를 의미하는 문서 번호와 해당 문서의 토픽 비중을 한 줄씩 꺼내온다.
    for i, topic_list in enumerate(ldamodel[corpus]):
        doc = topic_list[0] if ldamodel.per_word_topics else topic_list            
        doc = sorted(doc, key=lambda x: (x[1]), reverse=True)
        # 각 문서에 대해서 비중이 높은 토픽순으로 토픽을 정렬한다.
        # EX) 정렬 전 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (10번 토픽, 5%), (12번 토픽, 21.5%), 
        # Ex) 정렬 후 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (12번 토픽, 21.5%), (10번 토픽, 5%)
        # 48 > 25 > 21 > 5 순으로 정렬이 된 것.

        # 모든 문서에 대해서 각각 아래를 수행
        for j, (topic_num, prop_topic) in enumerate(doc): #  몇 번 토픽인지와 비중을 나눠서 저장한다.
            if j == 0:  # 정렬을 한 상태이므로 가장 앞에 있는 것이 가장 비중이 높은 토픽
                topic_table = topic_table.append(pd.Series([int(topic_num), round(prop_topic,4), topic_list]), ignore_index=True)
                # 가장 비중이 높은 토픽과, 가장 비중이 높은 토픽의 비중과, 전체 토픽의 비중을 저장한다.
            else:
                break
    return(topic_table)

In [21]:
topictable = make_topictable_per_doc(ldamodel, corpus)
topictable = topictable.reset_index() # 문서 번호을 의미하는 열(column)로 사용하기 위해서 인덱스 열을 하나 더 만든다.
topictable.columns = ['문서 번호', '가장 비중이 높은 토픽', '가장 높은 토픽의 비중', '각 토픽의 비중']
topictable[:10]

,문서 번호,가장 비중이 높은 토픽,가장 높은 토픽의 비중,각 토픽의 비중
0,0,2.0,0.5490,"[(2, 0.54904664), (6, 0.10314923), (9, 0.02102..."
1,1,2.0,0.4004,"[(2, 0.40039077), (3, 0.028745368), (4, 0.0990..."
2,2,2.0,0.7714,"[(2, 0.7714084), (6, 0.172917), (13, 0.0419292..."
3,3,2.0,0.3476,"[(2, 0.3476336), (3, 0.058245614), (12, 0.2963..."
4,4,0.0,0.3757,"[(0, 0.3757033), (2, 0.27073643), (5, 0.076105..."
5,5,19.0,0.4350,"[(2, 0.27132744), (16, 0.25671378), (19, 0.434..."
6,6,14.0,0.6194,"[(1, 0.16128916), (2, 0.016181415), (3, 0.0840..."
7,7,2.0,0.6556,"[(0, 0.020050058), (2, 0.6556459), (3, 0.23922..."
8,8,2.0,0.3948,"[(2, 0.39478636), (3, 0.08821386), (4, 0.30942..."
9,9,14.0,0.5697,"[(2, 0.28254935), (3, 0.020095252), (8, 0.0706..."
